In [1]:
import nltk  # pip install nltk
from os import listdir
import numpy as np  # pip install numpy
import pandas as pd  # pip install pandas
import matplotlib.pyplot as plt  # pip install matplotlib 
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy  # pip install spacy
from spacy import displacy 
from tqdm.notebook import trange, tqdm  # pip install tqdm

In [2]:
# The file structure should have the "news" folder containing the 3 .csv's in the same root folder as this noteboo

# Load the data from the disk into a pandas dataframe

dfs = []
for x in [1, 2, 3]:
    for chunk in pd.read_csv(f'./news/articles{x}.csv', chunksize = 100):
        dfs.append(chunk)
df = pd.concat(dfs, ignore_index=True)

In [ ]:
# Displays the content of a random article for testing purpose

df.loc[17290, 'content']

In [3]:
# Loads the spacy NER resource

nlp = spacy.load('en_core_web_sm')

In [6]:
doc = df.loc[1, 'content']
doc = nlp(doc)
displacy.render(doc, style='ent', jupyter=True)

In [5]:
df_name_entities = []
total = df.shape[0]
for i in trange(df.shape[0]):
    print(i, end='\r')
    doc = df.loc[i, 'content']
    article_id = df.loc[i, 'id']
    doc = nlp(doc)
    entities = [(e.text, e.label_) for e in doc.ents if len(e.text.strip(' --')) > 0 and e.label_ != 'CARDINAL']

    f = pd.DataFrame(entities)
    f['id'] = article_id
    df_name_entities.append(f)

  0%|          | 0/142570 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [8]:
nedf = pd.concat(df_name_entities, ignore_index=True)

In [9]:
nedf.columns = ['Text', 'Index_Start', 'Index_End', 'Type', 'Article_ID']

In [10]:
nedf.head(100)

,Text,Index_Start,Index_End,Type,Article_ID
0,WASHINGTON,0,10,GPE,17283
1,Congressional,16,29,NORP,17283
2,Republicans,30,41,NORP,17283
3,Obama,116,121,PERSON,17283
4,Trump,167,172,ORG,17283
...,...,...,...,...,...
95,the South Bronx,2293,2308,LOC,17284
96,Hispanic,2348,2356,NORP,17284
97,the United States,2382,2399,GPE,17284
98,hours,2565,2570,TIME,17284


In [34]:
nedf.to_csv('entities.csv')

In [2]:
dfs = []
for chunk in pd.read_csv(f'./entities.csv', chunksize = 1000):
    dfs.append(chunk)
        
df_entities = pd.concat(dfs, ignore_index=True)

In [3]:
from whoosh.index import create_in
from whoosh.fields import *

In [14]:
schema = Schema(text=TEXT(stored=True), entity_type=TEXT(stored=True), article_id=NUMERIC(stored=True))

In [15]:
import os.path
if not os.path.exists('index__'):
    os.mkdir('index__')
ix = create_in('index__', schema)

In [3]:
from whoosh.index import open_dir

In [4]:
ix = open_dir('index__')

In [17]:
writer = ix.writer()
sz = df_entities.shape[0]
for i in trange(sz):
    print(f'{i} of {sz}', end='\r')
    _text = str(df_entities.loc[i, 'Text'])
    _entity_type = df_entities.loc[i, 'Type']
    _article_id = df_entities.loc[i, 'Article_ID']
    writer.add_document(text=_text, entity_type=_entity_type, article_id=_article_id)
    
print(_text, _entity_type, _article_id)
writer.commit()

  0%|          | 0/8408848 [00:00<?, ?it/s]

Trump Police PERSON 218082


In [6]:
from whoosh.query import *
from whoosh.qparser import QueryParser

In [7]:
with ix.searcher() as searcher:
    parser = QueryParser('text', ix.schema)
    query = parser.parse(u'Trump')
    results = searcher.search(query)
    print(len(results))
    for res in results:
        print(res)

384493
<Hit {'article_id': 59193, 'entity_type': 'WORK_OF_ART', 'text': 'Trump Trump Trump Trump Trump'}>
<Hit {'article_id': 18756, 'entity_type': 'PERSON', 'text': 'Trump Trump Trump'}>
<Hit {'article_id': 43948, 'entity_type': 'PERSON', 'text': 'Trump Trump Trump'}>
<Hit {'article_id': 88574, 'entity_type': 'PERSON', 'text': 'Trump Trump Trump'}>
<Hit {'article_id': 17813, 'entity_type': 'WORK_OF_ART', 'text': 'Trump! Trump'}>
<Hit {'article_id': 36630, 'entity_type': 'WORK_OF_ART', 'text': 'Trump! Trump'}>
<Hit {'article_id': 36703, 'entity_type': 'WORK_OF_ART', 'text': 'Trump! Trump!'}>
<Hit {'article_id': 36866, 'entity_type': 'PERSON', 'text': 'Trump Trump'}>
<Hit {'article_id': 36949, 'entity_type': 'WORK_OF_ART', 'text': 'Trump! Trump!'}>
<Hit {'article_id': 37952, 'entity_type': 'WORK_OF_ART', 'text': 'Trump! Trump!'}>


In [13]:
df.loc[5913, 'Article_ID']

KeyError: 'Article_ID'